<a href="https://colab.research.google.com/github/lorransr/taxonomy_method/blob/main/Taxonomy_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Example

In [38]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform

In [188]:
#inputs
data = {"C1":[0.71,1.33,1.45],
        "C2":[4.1,5.9,4.9],
        "C3":[0.18,0.74,0.27]
        }
criteria_type = {"C1":"cost","C2":"cost","C3":"profit"}
index = ["A1","A2","A3"]
raw = pd.DataFrame(data,index=index)

## Passo 1: 
- Média e Desvio Padrão

$$\bar{r}_j = \frac{1}{m} \sum^m_{i=1}{r_{ij}}$$

$$S_j = \sqrt{\frac{1}{m}\sum^m_{i=1}{r_{ij}-\bar{r}_j}}$$

In [11]:
criteria_mean = raw.mean()
criteria_std = raw.std()

## Passo 2
- Matriz Normalizada

In [15]:
std_matrix = (raw - criteria_mean)/criteria_std

## Passo 3
- Matriz de distâncias compostas

In [76]:
#distância euclidiana entre A1 e A2
np.sqrt(((std_matrix.loc["A1"]-std_matrix.loc["A2"])**2).sum())

3.144579296487791

In [85]:
#distâncias euclidianas entre alternativas
pdist(std_matrix)

array([3.1445793 , 2.08521849, 1.93997033])

In [86]:
composite_distance = squareform(pdist(std_matrix))

composite_distance_df = pd.DataFrame(
    composite_distance,
    index=index,
    columns=index)
composite_distance_df

,A1,A2,A3
A1,0.000000,3.144579,2.085218
A2,3.144579,0.000000,1.939970
A3,2.085218,1.939970,0.000000


## Passo 4 
- Homogeneinização das alternativas

toma-se a média e desvio padrão das distâncias minimas de cada alternativa, de acordo com as equações.

$$\bar{O} = \frac{1}{m} \sum^m_{i=1}{r_{ij}}$$

$$S_o = \sqrt{\frac{1}{m}\sum^m_{i=1}{O_{i}-\bar{O}}}$$

A equação:

$$O = \bar{O} \pm 2S_o$$

determina a homogeneidade da matriz de distâncias compostas.

Se a distância mínima de cada alternativa não se encontra situada no intervalo, a alternativa é considerada inomogênea, e é então descartada. Em seguida recalcula-se os valores de média e desbio padrão das distâncias mínimas.

In [99]:
filter_1 = composite_distance_df>0
shortest_distance_values = composite_distance_df[filter_1].min()
shortest_distance_values

A1    2.085218
A2    1.939970
A3    1.939970
dtype: float64

In [138]:
mean_shortest = shortest_distance_values.mean()
std_shortest = shortest_distance_values.std()

print('{} +- 2 * {}\n'.format(mean_shortest,std_shortest))

lower = mean_shortest-2*std_shortest
upper = mean_shortest+2*std_shortest

accepted_range = [lower,upper]
print("accepted range: {}".format(accepted_range))

1.9883863858665825 +- 2 * 0.08385906248480482

accepted range: [1.8206682608969729, 2.156104510836192]


In [192]:
condition_1=shortest_distance_values >= accepted_range[0]
condition_2=shortest_distance_values <= accepted_range[1]

(condition_1) & (condition_2)

A1    True
A2    True
A3    True
dtype: bool

In [193]:
std_matrix_filtered = std_matrix.loc[shortest_distance_values[(condition_1) & (condition_2)].index]

## Passo 5
* O padrão de desenvolvimento

O padrão de desenvolvimento de atributos é determinado através da equação abaixo, utilizando-se a matriz obtida no passo 2.

$$C_{io} = \sqrt{\sum^n_{j=1}{(z_{ij}-z_{oj})^2}}$$

Onde, $Z_{oj}$ representa o valor ideal do j-ésimo atributo, a depender da característica monotônica de lucro ou custo.

In [194]:
ideal_values = std_matrix_filtered.apply(
    lambda x: x.max() if criteria_type[x.name]=="profit" else x.min()
    )
ideal_values

C1   -1.141447
C2   -0.960988
C3    1.141699
dtype: float64

In [196]:
development_pattern = np.sqrt(((std_matrix_filtered-ideal_values)**2).sum(axis=1))
development_pattern

A1    1.862189
A2    2.533896
A3    2.588677
dtype: float64

## Passo 6
- Ranking Final das Alternativas

O limite superior do desenvolvimento $C_{O}$ é inicialmente calculado de acordo com a equação:

$$C_o=\bar{C}_{io} + 2S_{C_{io}}$$

Por fim os atributos são ranqueados de a acordo com o atributo de desenvolvimento $F_{i}$ que diz respeito a i-gésima alternativa e é obtico a partir da equação:

$$F_{i}=\frac{C_{io}}{C_o}$$

O valor de $F_i$ é entre zero e um.

Valores próximos de 0 indicam um maior desenvolvimento da alternativa (rank maior).

Valores próximos de um demonstram uma falta de desenvolvimento da alternativa (rank menor)

In [197]:
high_limit = development_pattern.mean() + 2*development_pattern.std()
high_limit

3.1373596451972654

In [198]:
development_attribute = development_pattern/high_limit
development_attribute.sort_values()

A1    0.593553
A2    0.807653
A3    0.825113
dtype: float64